We start with some imports and initializing random with a seed.

In [1]:
import operator
import copy
import random
import scipy
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from skfusion import fusion
from os.path import join as pjoin
from scipy.stats import pearsonr
from scipy.stats import rankdata
from collections import defaultdict
from matplotlib import cm
from random import shuffle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

random.seed(42)

Normalization function. Performs cell-wise division of matrix $X$ using matrix $paths$.

In [2]:
def normalize(X, paths):
    normalized = np.zeros((X.shape[0], X.shape[1]), dtype='float')
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if paths[i, j] != 0:
                normalized[i, j] = X[i, j] / paths[i, j]
    return normalized

Evaluation functions

In [3]:
def evaluate(prediction, test_set, paths=None):
    pred = copy.deepcopy(prediction)
    if paths is None:
        #Change all known cells to -infinity
        pred[user_item > 0] = -np.inf # known
        rank = np.zeros(pred.shape)
        for i in range(pred.shape[0]):
            rank[i] = len(rank[i]) - rankdata(pred[i], method="min")
    else:  
        #Change value of known cells to 0 in both matrices
        predi = pred * (user_item-1)**2
        n_paths = paths * (user_item-1)**2
        rank = rank_pareto(predi, n_paths)
 
    #print(rank)
    res = []
    for a in test_set:
        res.append(rank[unique_users.index(a[0]), unique_items.index(a[1])])
    return np.array(res)

In [4]:
def recall(r, n):
    return (r < n).sum()/len(r)

In [5]:
def remove(lst, val):
    i = 0
    while val in lst:
        lst.remove(val)
        i += 1
    return i

def indices(lst, element):
    result = []
    offset = -1
    while True:
        try:
            offset = lst.index(element, offset+1)
        except ValueError:
            return result
        result.append(offset)

def rank_pareto(pred, paths):
    rank = np.ones(pred.shape)*np.inf
    for i in range(pred.shape[0]):
        r = 1
    #rank = np.zeros(user.shape)
        Xs, Ys = pred[i], paths[i]
        num = [[Xs[i], Ys[i]] for i in range(len(Xs))]

        maxX = True
        maxY = True
        myList = [[Xs[i], Ys[i]] for i in range(len(Xs))]
        myList.sort(key=operator.itemgetter(0,1), reverse=True) 
        while len(myList) > 0:
            if myList[0][0] > 0 and myList[0][1] > 0: # ignore cases where both are zero
                p_front = [myList[0]] 
                for pair in myList[1:]:
                    if pair[1] > p_front[-1][1]: # Look for higher values of Y…
                        p_front.append(pair) # … and add them to the Pareto frontier     
                p_frontX = [pair[0] for pair in p_front]
                p_frontY = [pair[1] for pair in p_front]
                #plt.plot(p_frontX, p_frontY); 
                new_r = r
                for pair in p_front:
                    new_r = new_r + remove(myList, pair)
                    for j in indices(num, pair):
                        rank[i,j] = r
                r = new_r
            else:
                break
            #plt.plot(Xs, Ys, ".");
    return rank

Function for DFMF

In [6]:
def dfmf(i, user_item, user_tag, item_tag):
    t_users = fusion.ObjectType("User", i)
    t_items = fusion.ObjectType("item", i)
    t_tags = fusion.ObjectType("Tag", i)

    user_item_ma = np.ma.masked_equal(user_item, 0)
    user_tag_ma = np.ma.masked_equal(user_tag, 0)
    item_tag_ma = np.ma.masked_equal(item_tag, 0)

    random_type = 'random_vcol'
    relations = [fusion.Relation(user_item_ma, t_users, t_items), 
                 fusion.Relation(user_tag_ma, t_users, t_tags), 
                 fusion.Relation(item_tag_ma.T, t_tags, t_items)]
    fusion_graph = fusion.FusionGraph()
    fusion_graph.add_relations_from(relations)
    fuser = fusion.Dfmf(init_type=random_type)
    fuser.fuse(fusion_graph);

    S_user_tag = fuser.backbone(fuser.fusion_graph[t_users][t_tags][0])
    S_tag_item = fuser.backbone(fuser.fusion_graph[t_tags][t_items][0])
    S_user_item = fuser.backbone(fuser.fusion_graph[t_users][t_items][0])
    S = [S_user_tag, S_tag_item, S_user_item]

    G_users = fuser.factor(t_users)
    G_items = fuser.factor(t_items)
    G_tags = fuser.factor(t_tags)
    G = [G_users, G_items, G_tags]
    
    return (G, S)

Function to create synthetic data

In [7]:
lim = 1000
def makeList(size, offset=0):
    A = random.sample(range(lim), size)
    A.sort()
    return A

def synthetic_data(group_fulness=1, intra_group=0):
    sizeU = 600
    sizeI = 120
    sizeT = 110

    U = makeList(sizeU)
    I = makeList(sizeI)
    T = makeList(sizeT)

    Rui =  np.zeros((len(U), len(I)), dtype='int')
    Rut =  np.zeros((len(U), len(T)), dtype='int')
    Rit =  np.zeros((len(I), len(T)), dtype='int')

    pairs = defaultdict(list)
    for i in range(sizeU):
        for j in range(sizeI):
            if U[i]//200 == I[j]//200:
                if np.random.rand() < group_fulness:
                    for k in range(sizeT):
                        ut = 0
                        it = 0
                        if U[i]//200 == T[k]//200:
                            if np.random.rand() < group_fulness:
                                ut = np.random.randn() + 1
                        else:
                            if np.random.rand() < intra_group:
                                ut = np.random.randn()
                        if I[j]//200 == T[k]//200:
                            if np.random.rand() < group_fulness:
                                it = np.random.randn() + 1
                        else:
                            if np.random.rand() < intra_group:
                                it = np.random.randn()
                        if ut * it >= 1:
                            pairs[U[i], I[j]].append(T[k])

            else:
                if np.random.rand() < intra_group:
                    for k in range(sizeT):
                        ut = 0
                        it = 0
                        if U[i]//200 == T[k]//200:
                            if np.random.rand() < group_fulness:
                                ut = np.random.randn() + 1
                        else:
                            if np.random.rand() < intra_group:
                                ut = np.random.randn() - 1
                        if I[j]//200 == T[k]//200:
                            if np.random.rand() < group_fulness:
                                it = np.random.randn() + 1
                        else:
                            if np.random.rand() < intra_group:
                                it = np.random.randn() - 1
                        if ut * it >= 1:
                            pairs[U[i], I[j]].append(T[k])

    entries = list(pairs.keys())
    np.random.shuffle(entries)

    unique_users = list(U)
    unique_items = list(I)
    unique_tags = list(T)

    user_item = np.zeros((len(unique_users), len(unique_items)))
    user_tag = np.zeros((len(unique_users), len(unique_tags)))
    item_tag = np.zeros((len(unique_items), len(unique_tags)))

    i = 0
    for a in entries:
        user_item[unique_users.index(a[0]), unique_items.index(a[1])] = 1
        for entry in pairs[(a[0], a[1])]:
            user_tag[unique_users.index(a[0]), unique_tags.index(entry)] += 1
            item_tag[unique_items.index(a[1]), unique_tags.index(entry)] += 1


    fig = plt.figure(figsize=(720*1./100, 230*1./100))
    gs = gridspec.GridSpec(2, 4,
                           width_ratios=[600, 5, 110, 5], #    110, 120],
                           height_ratios=[120, 110] #    600, 120]
                           )
    gs.update(left=0.0, right=1.0) #, wspace=0.1, hspace=0.1)

    plt.subplots_adjust(bottom=0, top=1, left=0, right=0.1,wspace = 0.2)

    ax = plt.subplot(gs[0])
    plt.ylabel("ITEM")
    plt.xticks([])
    plt.yticks([]) 
    im2 = plt.imshow(user_item.T, cmap=plt.cm.Reds, aspect="equal")
    plt.xlabel("USER")
    ax.xaxis.set_label_position('top') 
    ticks = [0, user_item.max()]
    #plt.colorbar(im2, shrink=0.5, pad=0.02, aspect=10, ticks=ticks)

    ax = plt.subplot(gs[1])
    plt.colorbar(im2, cax=ax, ticks=ticks)
    v = ax.get_position()
    v.x0 -= 0.025
    v.x1 -= 0.025
    v.y0 += 0.1
    v.y1 -= 0.1
    ax.set_position(v)

    ax = plt.subplot(gs[2])
    plt.ylabel("ITEM")  
    plt.xticks([])
    plt.yticks([]) 
    plt.xlabel('TAG')
    ax.xaxis.set_label_position('top') 
    im3 = plt.imshow(item_tag, cmap=plt.cm.Greens, aspect="equal")
    ticks = [0, item_tag.max()]
    #plt.colorbar(im3, shrink=0.5, pad=0.02, aspect=10, ticks=ticks)

    ax = plt.subplot(gs[3])
    plt.colorbar(im3, cax=ax, ticks=ticks)
    v = ax.get_position()
    v.x0 -= 0.025
    v.x1 -= 0.025
    v.y0 += 0.1
    v.y1 -= 0.1
    ax.set_position(v)


    ax = plt.subplot(gs[4])
    plt.ylabel("TAG")
    plt.xticks([])
    plt.yticks([]) 
    im1 = plt.imshow(user_tag.T, cmap=plt.cm.Blues, aspect="equal")
    plt.xlabel("USER")
    ax.xaxis.set_label_position('top') 
    ticks = [0, user_tag.max()]
    #plt.colorbar(im1, shrink=0.5, pad=0.02, aspect=10, ticks=ticks)

    ax = plt.subplot(gs[5])
    plt.colorbar(im1, cax=ax, ticks=ticks)
    v = ax.get_position()
    v.x0 -= 0.025
    v.x1 -= 0.025
    v.y0 += 0.1
    v.y1 -= 0.1
    ax.set_position(v)


    train_set = entries[:int(len(entries) * 0.75)]
    test_set = entries[int(len(entries) * 0.75):]
    return unique_users, unique_tags, unique_items, pairs, train_set, test_set

New paths - filtered matrix-wise

In [8]:
def importantCells(x, m):
    x_u_p = np.zeros(x.shape)
    tmp = np.copy(x)
    tmp = tmp.flatten()
    tmp.sort()
    return (x >= tmp[int(m*len(tmp)/100)-1])*1.

In [9]:
def matrixFilter(b, d, contour=True):
    a = np.zeros((20, 20))
    filtered = np.zeros((20, 20))
    pareto = np.zeros((20, 20))
    filtered_pareto = np.zeros((20, 20))
    for i, m in enumerate(range(5,101,5)):
        for j, n in enumerate(range(5,101,5)):
            b_u_p = importantCells(b, m)
            d_u_p = importantCells(d, n)
            paths = b_u_p.dot(d_u_p)
            res = recall(evaluate(paths, test_set), 20)
            a[i,j] = res
            
            B = b * b_u_p
            D = d * d_u_p
            r = B.dot(D)
            res = recall(evaluate(r, test_set), 20)
            filtered[i,j] = res

            res = recall(evaluate(b.dot(d), test_set, paths=paths), 20)
            pareto[i,j] = res

            res = recall(evaluate(r, test_set, paths=paths), 20)
            filtered_pareto[i,j] = res
            
    if contour:
        plt.yticks([0,4,9,14,19], [5,25,50,75,100])
        plt.xticks([0,4,9,14,19], [5,25,50,75,100])
        cp = plt.contourf(a, 5, cmap=cm.gray.reversed())
        plt.colorbar(cmap=cp);
    else:
        #heatmap    
        a = np.flipud(a) 
        plt.yticks([0,4,9,14,19], [100,75,50,25,5])
        plt.xticks([0,4,9,14,19], [5,25,50,75,100])
        hm = plt.imshow(a, cmap='hot', vmax=1);
        plt.colorbar(hm);

    print("Maximum paths: " , a.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(a.argmax(), a.shape)])
    print("Maximum filtered values: " , filtered.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(filtered.argmax(), a.shape)])
    print("Maximum pareto: " , pareto.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(pareto.argmax(), a.shape)])
    print("Maximum pareto on filtered: " , filtered_pareto.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(filtered_pareto.argmax(), a.shape)])
    return (a.max(), filtered.max(), pareto.max(), filtered_pareto.max())

New paths - filtered row/column-wise

In [10]:
def importantCells2(x, m, axis=1):
    x_u_p = np.zeros(x.shape)
    tmp = np.copy(x)
    tmp.sort(axis=axis)
    if axis:
        for i in range(x.shape[0]):
            x_u_p[i] = (x[i] >= tmp[i][int(m*len(tmp[i])/100)-1]) * 1.
    else:      
        for i in range(x.shape[1]):
            x_u_p[:,i] = (x[:,i] >= tmp[:,i][int(m*len(tmp[:,i])/100)-1]) * 1.
    return x_u_p

In [11]:
def rowFilter(b, d, contour=True):
    a = np.zeros((20, 20))
    filtered = np.zeros((20, 20))
    pareto = np.zeros((20, 20))
    filtered_pareto = np.zeros((20, 20))
    for k, m in enumerate(range(5,101,5)):
        for l, n in enumerate(range(5,101,5)):        
            b_u_p = np.zeros(b.shape)
            tmp = np.copy(b)
            tmp.sort(axis=1)
            for i in range(b.shape[0]):
                b_u_p[i] = (b[i] >= tmp[i][int(m*len(tmp[i])/100)-1]) * 1.

            d_u_p = np.zeros(d.shape)
            tmp = np.copy(d)
            tmp.sort(axis=0)
            for i in range(d.shape[1]):
                d_u_p[:,i] = (d[:,i] >= tmp[:,i][int(n*len(tmp[:,i])/100)-1]) * 1.

            paths = b_u_p.dot(d_u_p)
            res = recall(evaluate(paths, test_set), 20)
            a[k,l] = res
            
            B = b * b_u_p
            D = d * d_u_p
            r = B.dot(D)
            res = recall(evaluate(r, test_set), 20)
            filtered[k,l] = res

            res = recall(evaluate(b.dot(d), test_set, paths=paths), 20)
            pareto[k,l] = res
            res = recall(evaluate(r, test_set, paths=paths), 20)
            filtered_pareto[k,l] = res
    if contour:
        plt.yticks([0,4,9,14,19], [5,25,50,75,100])
        plt.xticks([0,4,9,14,19], [5,25,50,75,100])
        cp = plt.contourf(a, 5, cmap=cm.gray.reversed())
        plt.colorbar(cmap=cp);
    else:
        #heatmap    
        a = np.flipud(a) 
        plt.yticks([0,4,9,14,19], [100,75,50,25,5])
        plt.xticks([0,4,9,14,19], [5,25,50,75,100])
        hm = plt.imshow(a, cmap='hot', vmax=1);
        plt.colorbar(hm);
    print("Maximum paths: " , a.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(a.argmax(), a.shape)])
    print("Maximum filtered values: " , filtered.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(filtered.argmax(), a.shape)])
    print("Maximum pareto: " , pareto.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(pareto.argmax(), a.shape)])
    print("Maximum pareto on filtered: " , filtered_pareto.max() , ", with filters: ", 
          [(index + 1) * 5 for index in np.unravel_index(filtered_pareto.argmax(), a.shape)])
    return (a.max(), filtered.max(), pareto.max(), filtered_pareto.max())